# Improved RAG for Single Document Q&A

This notebook demonstrates how to:
1. Convert a PDF document to text
2. Split the text into page-level chunks
3. Use Highlights API to search for relevant chunks
4. Send the most relevant chunks to OpenAI for text generation

## Setup

First, let's install and import the required libraries.

In [ ]:
!pip install PyPDF2 openai python-dotenv

import os
import PyPDF2
import openai
from dotenv import load_dotenv
from typing import List, Dict
from base_client import HighlightsClient
from utils import PDFProcessor

## Loading Environment Variables

Create a .env file with your API keys:
```
HIGHLIGHTS_API_KEY=your-highlights-api-key
OPENAI_API_KEY=your-openai-api-key
```

In [3]:
# Load environment variables
load_dotenv()

# Initialize clients
highlights_client = HighlightsClient(api_key=os.getenv('HIGHLIGHTS_API_KEY'))
openai.api_key = os.getenv('OPENAI_API_KEY')

## Using the PDF Processor

Now let's try processing a PDF document.

In [4]:
# Initialize processor
processor = PDFProcessor(highlights_client)

# Path to your PDF file
pdf_path = 'data/border_act.pdf'

# Extract text chunks from PDF
text_chunks = processor.extract_text_from_pdf(pdf_path)

print(f"Extracted {len(text_chunks)} pages from PDF")
print("\nSample from first page:")
print(text_chunks[0][:200] + "...")

Extracted 292 pages from PDF

Sample from first page:
II 
Calendar No. 397 
118 THCONGRESS 
2DSESSION  S. 4361 
Making emergency supplemental appropriations for border security and com-
batting fentanyl for the fiscal year ending September 30, 2024, and ...


## Utilizing Highlights to retrieve relevant chunks

Let's search for relevant chunks given the query.

In [5]:
query = "Am I an eligible individual for CONDITIONAL PERMANENT RESIDENT STATUS? I was paroled into the us in 2020."

# Search for relevant chunks
relevant_chunks = processor.search_relevant_chunks(
    query=query,
    text_chunks=text_chunks,
    top_n=5
)

print(f"Found {len(relevant_chunks)} relevant chunks")


Found 5 relevant chunks


# Using OpenAI with Highlights response
Highlights returns the original chunks that we passed in so forwarding those into OpenAI 

In [6]:
# Generate response using OpenAI
response = processor.generate_response(
    query=query,
    context=relevant_chunks
)

print("\nGenerated Response:")
print(response)


Generated Response:
Based on the context provided, to be considered an "eligible individual" for CONDITIONAL PERMANENT RESIDENT STATUS, you must meet the following criteria:

1. You must be present in the United States.
2. You must be a citizen or national of Afghanistan, or if you have no nationality, you must have last habitually resided in Afghanistan.
3. You must not have been granted permanent resident status.
4. You must have been inspected and admitted to the United States on or before the date of the enactment of the Act, or you must have been paroled into the United States during the period beginning on July 30, 2021, and ending on the date of the enactment of the Act, provided that such parole has not been terminated.

Since you mentioned that you were paroled into the U.S. in 2020, it is essential to note that this is before the specified period of July 30, 2021, which means you would not qualify under the fourth criterion. 

Therefore, based on the information you provided